In [101]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [102]:
import tensorflow as tf
print(tf.__version__)
import string
import re
import nltk
from os import listdir
from collections import Counter
from numpy import array
import numpy as np
import pandas as pd
from nltk.corpus import stopwords

2.12.0


In [103]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [104]:
!pip install --upgrade tensorflow-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached tensorflow-gpu-2.12.0.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [105]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [106]:
# turn a doc into clean tokens
def clean_doc(doc):
# split into tokens by white space
    tokens = doc.split()
# prepare regex for char filtering
    re_punc = re.compile( ' [%s] ' % re.escape(string.punctuation))
# remove punctuation from each word
    tokens = [re_punc.sub( '' , w) for w in tokens]
# remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
# filter out stop words
    stop_words = set(stopwords.words( 'english' ))
    tokens = [w for w in tokens if not w in stop_words]
# filter out short tokens
    tokens = [word for word in tokens if len(word) > 1]
    return tokens


In [107]:
# Load the dataset
train_df = pd.read_csv("/content/drive/MyDrive/NEEWWWWW/DATA_NEW/PC/500/train_400.csv")
test_df = pd.read_csv("/content/drive/MyDrive/NEEWWWWW/DATA_NEW/PC/500/test_100.csv")


In [108]:
train_df[['text','label']].head()

,text,label
0,"Easy to use, economical!",1
1,Digital is where it's at...down with d...,1
2,"Good image quality, 3x optical zoom, m...",1
3,Awesome features/easy to use/fun/versa...,1
4,simple to use,1


In [109]:
train_df['label'].value_counts()

1    200
0    200
Name: label, dtype: int64

In [110]:
train_pos_sentences = train_df['text'].loc[train_df.label==1]
train_neg_sentences = train_df['text'].loc[train_df.label==0]
train_pos_sentences = train_pos_sentences.reset_index(drop=True)
train_neg_sentences = train_neg_sentences.reset_index(drop=True)

test_pos_sentences = test_df['text'].loc[test_df.label==1]
test_neg_sentences = test_df['text'].loc[test_df.label==0]
test_pos_sentences = test_pos_sentences.reset_index(drop=True)
test_neg_sentences = test_neg_sentences.reset_index(drop=True)

In [111]:
print(train_pos_sentences[0:5].values)

['        Easy to use, economical!'
 "        Digital is where it's at...down with developing film!"
 '        Good image quality, 3x optical zoom, macro mode, inexpensive'
 '        Awesome features/easy to use/fun/versatile/low price/Cust SVS 2nd 2 none!'
 '        simple to use']


In [112]:
print(train_neg_sentences[0:5].values)

['        LCD hard to see in the sun, LCD freezes'
 '        Eats batteries, plan on shelling out more money, Not very high res'
 '        Very Poor customer service.'
 '        The camera I have can only be connected to your computer via a serial port, and as I have a serial mouse and a serial modem, I have no spare ports while online, forcing me to disconnect to upload new pictures. :-( [Update: I now have a PS/2 mouse and can p'
 '        Potentially weak LCD screen']


In [113]:
text = train_pos_sentences[0]
tokens = clean_doc(text)
print(tokens)

['Easy']


In [114]:
# load doc and add to vocab
def add_sent_to_vocab(text,vocab):
# clean text
    tokens = clean_doc(text)
# update counts
    vocab.update(tokens)

def process_docs_to_vocab(sentList, vocab):
    for i in range(len(sentList)):
        text = sentList[i]
        add_sent_to_vocab(text,vocab)
       
# turn a doc into clean tokens
def clean_doc_wVocab(doc, vocab):
	# split into tokens by white space
	tokens = doc.split()
	# prepare regex for char filtering
	re_punc = re.compile('[%s]' % re.escape(string.punctuation))
	# remove punctuation from each word
	tokens = [re_punc.sub('', w) for w in tokens]
	# filter out tokens not in vocab
	tokens = [w for w in tokens if w in vocab]
	tokens = ' '.join(tokens)
	return tokens

# load all docs in a directory, into tokens
def process_docs_to_tokens(sentList, vocab):
	documents = list()
	# walk through all sentences
	for i in range(len(sentList)):
		doc = sentList[i]
		# clean doc
		tokens = clean_doc_wVocab(doc, vocab)
		# add to list
		documents.append(tokens)
	return documents

# integer encode and pad documents
def encode_docs(tokenizer, max_length, docs):
    # integer encode
    encoded = tokenizer.texts_to_sequences(docs)
    # pad sequences
    padded = tf.keras.preprocessing.sequence.pad_sequences(encoded, maxlen=max_length, padding='post')
    return padded

# fit a tokenizer
def create_tokenizer(lines):
	tokenizer = tf.keras.preprocessing.text.Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer

In [115]:
# define vocab
vocab = Counter()
# add all docs to vocab
process_docs_to_vocab(train_df['text'], vocab)
process_docs_to_vocab(test_df['text'], vocab)
print(len(vocab))
# print the top words in the vocab
print(vocab.most_common(50))

813
[('quality', 61), ('good', 48), ('easy', 40), ('use', 32), ('Easy', 30), ('great', 28), ('low', 22), ('Good', 20), ('image', 20), ('camera', 19), ('battery', 19), ('pictures', 18), ('picture', 18), ('No', 18), ('Great', 17), ('software', 16), ('poor', 16), ('flash', 14), ('LCD', 13), ('USB', 13), ('high', 12), ('small', 12), ('batteries', 12), ('price', 11), ('print', 11), ('Poor', 11), ('bad', 11), ('inexpensive', 10), ('get', 10), ('hard', 10), ('cheap', 9), ('resolution', 9), ('Low', 9), ('lot', 9), ('Not', 9), ('simple', 8), ('fast', 8), ('digital', 8), ('Battery', 8), ('Picture', 7), ('customer', 7), ('Very', 7), ('Excellent', 7), ('photo', 7), ('memory', 7), ('Ease', 6), ('You', 6), ('features', 6), ('enough', 6), ('takes', 6)]


In [116]:
# load and clean a dataset
def load_clean_dataset(vocab):
	# load documents
	neg = process_docs_to_tokens(train_neg_sentences, vocab)
	pos = process_docs_to_tokens(train_pos_sentences, vocab)
	docs = neg + pos
	# prepare labels
	labels = array([0 for _ in range(len(neg))] + [1 for _ in range(len(pos))])
	return docs, labels

# load and clean a dataset
def load_clean_dataset_test(vocab):
	# load documents
	neg = process_docs_to_tokens(test_neg_sentences, vocab)
	pos = process_docs_to_tokens(test_pos_sentences, vocab)
	docs = neg + pos
	# prepare labels
	labels = array([0 for _ in range(len(neg))] + [1 for _ in range(len(pos))])
	return docs, labels

In [117]:
# load training data
train_docs, ytrain = load_clean_dataset(vocab)
# create the tokenizer
tokenizer = create_tokenizer(train_docs)
# define vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary size: %d' % vocab_size)
# calculate the maximum sequence length
max_length = max([len(s.split()) for s in train_docs])
print('Maximum length: %d' % max_length)

Vocabulary size: 619
Maximum length: 17


In [118]:
print(train_df['text'][0])

        Easy to use, economical!


In [119]:
train_docs[0]

'LCD hard see LCD freezes'

In [120]:
Xtrain = encode_docs(tokenizer, max_length, train_docs)

In [121]:
print(Xtrain.shape)
print(Xtrain[0])

(400, 17)
[ 22  26  95  22 266   0   0   0   0   0   0   0   0   0   0   0   0]


In [122]:
t = tokenizer
print(t.word_counts)
#print(t.document_count)
#print(t.word_index)
#print(t.word_docs)

OrderedDict([('lcd', 15), ('hard', 13), ('see', 4), ('freezes', 1), ('eats', 5), ('batteries', 12), ('plan', 3), ('shelling', 1), ('money', 6), ('not', 8), ('high', 16), ('res', 1), ('very', 6), ('poor', 28), ('customer', 5), ('service', 6), ('the', 4), ('camera', 21), ('connected', 1), ('computer', 3), ('via', 1), ('serial', 5), ('mouse', 2), ('spare', 1), ('ports', 1), ('forcing', 1), ('disconnect', 1), ('upload', 1), ('new', 1), ('pictures', 31), ('potentially', 1), ('weak', 2), ('screen', 4), ('slightly', 2), ('dated', 1), ('technology', 1), ('no', 12), ('flash', 32), ('rechargeable', 1), ('must', 2), ('battery', 20), ('life', 6), ('limited', 3), ('use', 52), ('mac', 3), ('compatability', 1), ('go', 2), ('dead', 2), ('short', 5), ('time', 7), ('consumption', 1), ('uses', 2), ('lot', 7), ('optical', 4), ('zoom', 3), ('bulky', 4), ('odd', 1), ('shutter', 2), ('slow', 3), ('low', 30), ('does', 2), ('meet', 1), ('advertised', 1), ('tiny', 5), ('button', 2), ('people', 1), ('large', 6),

In [123]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [124]:
# import tensorflow as tf
# from transformers import TFAutoModel, AutoTokenizer

# def define_model(length, vocab_size):
#     # channel 1
#     inputs1 = tf.keras.layers.Input(shape=(length,))
#     embedding1 = tf.keras.layers.Embedding(vocab_size, 100)(inputs1)
#     conv1 = tf.keras.layers.Conv1D(filters=32, kernel_size=4, activation='relu')(embedding1)
#     drop1 = tf.keras.layers.Dropout(0.5)(conv1)
#     pool1 = tf.keras.layers.MaxPooling1D(pool_size=2)(drop1)
#     flat1 = tf.keras.layers.Flatten()(pool1)
#     # channel 2
#     inputs2 = tf.keras.layers.Input(shape=(length,))
#     tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
#     bert_model = TFAutoModel.from_pretrained('bert-base-uncased')
#     input_ids = tf.keras.layers.Input(shape=(length,), dtype=tf.int32, name="input_ids")
#     bert_embedding = bert_model(input_ids)[0]
#     pooled_output = bert_embedding[:, 0, :]
#     dense2 = tf.keras.layers.Dense(100, activation='relu')(pooled_output)
#     # channel 3
#     inputs3 = tf.keras.layers.Input(shape=(length,))
#     embedding3 = tf.keras.layers.Embedding(vocab_size, 100)(inputs3)
#     lstm3 = tf.keras.layers.LSTM(64)(embedding3)
#     dense3 = tf.keras.layers.Dense(100, activation='relu')(lstm3)
#     # merge
#     merged = tf.keras.layers.concatenate([flat1, dense2, dense3])
#     # interpretation
#     dense4 = tf.keras.layers.Dense(10, activation='relu')(merged)
#     outputs = tf.keras.layers.Dense(1, activation='sigmoid')(dense4)
#     model = tf.keras.models.Model(inputs=[inputs1, input_ids, inputs3], outputs=outputs)
#     # compile
#     model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#     # summarize
#     print(model.summary())
#     return model

In [125]:
import tensorflow as tf
from transformers import TFAutoModel, AutoTokenizer
from gensim.models import KeyedVectors

def define_model(length, vocab_size, word2vec_model_path):
    # channel 1
    inputs1 = tf.keras.layers.Input(shape=(length,), dtype=tf.int32)
    tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
    input_ids = tf.keras.layers.Input(shape=(length,), dtype=tf.int32)
    bert_model = TFAutoModel.from_pretrained('bert-base-uncased')
    bert_output = bert_model(input_ids)[0] # use only the first tensor returned by the BERT model
    pooled_output = bert_output[:, 0, :]
    dense1 = tf.keras.layers.Dense(100, activation='relu')(pooled_output)

    # channel 2
    inputs2 = tf.keras.layers.Input(shape=(length,))
    word2vec_model = KeyedVectors.load_word2vec_format(word2vec_model_path, binary=True)
    vocab_size = min(vocab_size, len(word2vec_model.key_to_index)) # adjust vocab_size based on actual vocabulary size
    embedding_matrix = np.zeros((vocab_size, word2vec_model.vector_size))
    for word, index in word2vec_model.key_to_index.items():
        if index >= vocab_size:
            continue
        embedding_matrix[index] = word2vec_model[word]
    embedding2 = tf.keras.layers.Embedding(vocab_size, word2vec_model.vector_size, weights=[embedding_matrix], trainable=False)(inputs2)
    conv2 = tf.keras.layers.Conv1D(filters=32, kernel_size=4, activation='relu')(embedding2)
    drop2 = tf.keras.layers.Dropout(0.5)(conv2)
    pool2 = tf.keras.layers.MaxPooling1D(pool_size=2)(drop2)
    flat2 = tf.keras.layers.Flatten()(pool2)

    # channel 3
    inputs3 = tf.keras.layers.Input(shape=(length,))
    embedding3 = tf.keras.layers.Embedding(vocab_size, 100)(inputs3)
    lstm3 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64))(embedding3)
    dense3 = tf.keras.layers.Dense(100, activation='relu')(lstm3)

    # merge
    merged = tf.keras.layers.concatenate([dense1, flat2, dense3])

    # interpretation
    dense4 = tf.keras.layers.Dense(10, activation='relu')(merged)
    outputs = tf.keras.layers.Dense(1, activation='sigmoid')(dense4)

    model = tf.keras.models.Model(inputs=[input_ids, inputs2, inputs3], outputs=outputs)

    # compile
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    # summarize
    print(model.summary())

    return model

WORD2VEC_MODEL = '/content/drive/MyDrive/NEEWWWWW/GoogleNews-vectors-negative300.bin'
model = define_model(length=100, vocab_size=10000, word2vec_model_path=WORD2VEC_MODEL)

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_27 (InputLayer)          [(None, 100)]        0           []                               
                                                                                                  
 embedding_12 (Embedding)       (None, 100, 300)     3000000     ['input_27[0][0]']               
                                                                                                  
 input_26 (InputLayer)          [(None, 100)]        0           []                               
                                                                                                  
 conv1d_6 (Conv1D)              (None, 97, 32)       38432       ['embedding_12[0][0]']           
                                                                                            

In [126]:
word2vec_model_path = '/content/drive/MyDrive/NEEWWWWW/GoogleNews-vectors-negative300.bin'
print(vocab_size)
print(max_length)
# define model
model = define_model(max_length, vocab_size, word2vec_model_path)

619
17


Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_31 (InputLayer)          [(None, 17)]         0           []                               
                                                                                                  
 embedding_14 (Embedding)       (None, 17, 300)      185700      ['input_31[0][0]']               
                                                                                                  
 input_30 (InputLayer)          [(None, 17)]         0           []                               
                                                                                                  
 conv1d_7 (Conv1D)              (None, 14, 32)       38432       ['embedding_14[0][0]']           
                                                                                            

In [127]:
test_docs, ytest = load_clean_dataset_test(vocab)

In [128]:
Xtest = encode_docs(tokenizer, max_length, test_docs)

In [129]:
# # fit model
# model.fit([Xtrain,Xtrain,Xtrain], ytrain, batch_size=32, epochs=5)

In [130]:
# fit model
model.fit([Xtrain,Xtrain,Xtrain], ytrain, validation_data=([Xtest,Xtest,Xtest], ytest), batch_size=6, epochs=30)

Epoch 1/30


67/67 [==============================] - 69s 250ms/step - loss: 0.7233 - accuracy: 0.4950 - val_loss: 0.9964 - val_accuracy: 0.5000
Epoch 2/30
67/67 [==============================] - 11s 171ms/step - loss: 0.6994 - accuracy: 0.4975 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 3/30
67/67 [==============================] - 7s 108ms/step - loss: 0.6932 - accuracy: 0.4875 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 4/30
67/67 [==============================] - 7s 109ms/step - loss: 0.6607 - accuracy: 0.6225 - val_loss: 0.5389 - val_accuracy: 0.7900
Epoch 5/30
67/67 [==============================] - 6s 84ms/step - loss: 0.2326 - accuracy: 0.9300 - val_loss: 0.4270 - val_accuracy: 0.8500
Epoch 6/30
67/67 [==============================] - 8s 117ms/step - loss: 0.0714 - accuracy: 0.9700 - val_loss: 0.4774 - val_accuracy: 0.8700
Epoch 7/30
67/67 [==============================] - 6s 84ms/step - loss: 0.0302 - accuracy: 0.9900 - val_loss: 0.6531 - val_accuracy: 0.8800
Epoch 8/30
67/67 

In [131]:
# evaluate model on training dataset
_, acc = model.evaluate([Xtrain,Xtrain,Xtrain], ytrain, verbose=0)
print(' Train Accuracy: %f' % (acc*100))

 Train Accuracy: 100.000000


In [132]:
# evaluate model on test dataset
_, acc = model.evaluate([Xtest,Xtest,Xtest], ytest, verbose=0)
print( ' Test Accuracy: %f ' % (acc*100))
     

 Test Accuracy: 88.000000 


In [133]:
model.save('/content/gdrive/My Drive/sent_model_multi_input.h5')